# Feauture Selection

## Imports & Load Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [106]:
import numpy as np
import pandas as pd

import pickle

import mlflow
import mlflow.sklearn

In [3]:
# Cargar endpoint
infile = open('./data/21_ETL_tabular.pkl','rb')
tabular_ft = pickle.load(infile)
infile.close()

In [4]:
filename = './data/221_graph_metrics.pkl'
infile = open(filename, 'rb')
gmetrics_ft = pickle.load(infile)
infile.close()

In [5]:
filename = './data/222_ETRD_ROUTES_ELEC.pkl'
infile = open(filename, 'rb')
path_ft = pickle.load(infile)
infile.close()

In [6]:
filename = './data/223_ConsumerDensity.pkl'
infile = open(filename, 'rb')
con_cer_ft = pickle.load(infile)
infile.close()

In [7]:
filename = './data/1_OUTFT_UNTRD_FIC.pkl'
infile = open(filename, 'rb')
DEPVAR = pickle.load(infile)
infile.close()

## Preparar Datos

In [8]:
# Unir features en un unico dataframe
data = pd.merge(tabular_ft, gmetrics_ft, on = 'COD_ID')
data = pd.merge(data, path_ft, on = 'COD_ID')

# Adicionar variable dependiente FIC
data = pd.merge(data, DEPVAR.reset_index(), left_on='COD_ID', right_on='UNI_TR_D')

In [9]:
data.columns

Index(['COD_ID', 'FAS_CON_P', 'FAS_CON_S', 'FAS_CON_T', 'TEN_LIN_SE',
       'CAP_ELO', 'CAP_CHA', 'CONF', 'POSTO', 'POT_NOM', 'PER_FER', 'PER_TOT',
       'TIP_TRAFO', 'ARE_LOC', 'UNI_TR_D_x', 'CLAS_SUB', 'CNAE', 'TIP_CC',
       'TEN_FORN', 'GRU_TAR', 'CAR_INST', 'UCBT_ENE_MED', 'UCBT_ENE_STD',
       'UCBT_ENE_MAX', 'NCON', 'NCON_URB', 'CTMT_ENE_MED', 'CTMT_ENE_STD',
       'CTMT_ENE_MAX', 'PERD_A3a', 'PERD_A4', 'PERD_B', 'PERD_MED',
       'PERD_A3a_B', 'PERD_A4_B', 'PERD_B_A3a', 'PERD_B_A4', 'CTMT_PNTMT_MED',
       'CTMT_PNTMT_STD', 'CTMT_PNTMT_MAX', 'CTMT_PNTBT_MED', 'CTMT_PNTBT_STD',
       'CTMT_PNTBT_MAX', 'POT_NOM_TRS', 'POT_F01', 'POT_F02', 'PER_FER_TRS',
       'PER_TOT_TRS', 'TIP_TRAFO_TRS', 'UNTRS_ENES_MED', 'UNTRS_ENES_STD',
       'UNTRS_ENES_MAX', 'UNTRS_ENET_MED', 'UNTRS_ENET_STD', 'UNTRS_ENET_MAX',
       'DEGREE', 'NEIG_DEGREE', 'BET_CEN', 'CLO_CEN', 'PAGE_RANK',
       'DISTANCE_SUB', 'FCONV_R', 'I_MAX_MED_R', 'I_MAX_MIN_R', 'I_NOM_MED_R',
       'I_NOM_MIN_R', 'R

In [10]:
# Remove all rows with NAs
data.dropna(inplace=True)

# Isolate target
target = data.FIC

# Remove Keys and Target from dataframe
data.drop(['COD_ID','UNI_TR_D_x','UNI_TR_D_y', 'FIC'], axis=1, inplace=True)

### Feature clasification from type and origin

In [11]:
data.columns

Index(['FAS_CON_P', 'FAS_CON_S', 'FAS_CON_T', 'TEN_LIN_SE', 'CAP_ELO',
       'CAP_CHA', 'CONF', 'POSTO', 'POT_NOM', 'PER_FER', 'PER_TOT',
       'TIP_TRAFO', 'ARE_LOC', 'CLAS_SUB', 'CNAE', 'TIP_CC', 'TEN_FORN',
       'GRU_TAR', 'CAR_INST', 'UCBT_ENE_MED', 'UCBT_ENE_STD', 'UCBT_ENE_MAX',
       'NCON', 'NCON_URB', 'CTMT_ENE_MED', 'CTMT_ENE_STD', 'CTMT_ENE_MAX',
       'PERD_A3a', 'PERD_A4', 'PERD_B', 'PERD_MED', 'PERD_A3a_B', 'PERD_A4_B',
       'PERD_B_A3a', 'PERD_B_A4', 'CTMT_PNTMT_MED', 'CTMT_PNTMT_STD',
       'CTMT_PNTMT_MAX', 'CTMT_PNTBT_MED', 'CTMT_PNTBT_STD', 'CTMT_PNTBT_MAX',
       'POT_NOM_TRS', 'POT_F01', 'POT_F02', 'PER_FER_TRS', 'PER_TOT_TRS',
       'TIP_TRAFO_TRS', 'UNTRS_ENES_MED', 'UNTRS_ENES_STD', 'UNTRS_ENES_MAX',
       'UNTRS_ENET_MED', 'UNTRS_ENET_STD', 'UNTRS_ENET_MAX', 'DEGREE',
       'NEIG_DEGREE', 'BET_CEN', 'CLO_CEN', 'PAGE_RANK', 'DISTANCE_SUB',
       'FCONV_R', 'I_MAX_MED_R', 'I_MAX_MIN_R', 'I_NOM_MED_R', 'I_NOM_MIN_R',
       'REAC_SUB', 'RESI_SUB', 'Z

In [12]:
categorical = set(data.describe(include ='O').columns)

In [13]:
print(categorical)

{'TIP_TRAFO', 'GRU_TAR', 'CNAE', 'TEN_FORN', 'FAS_CON_T', 'CLAS_SUB', 'CAP_ELO', 'TIP_TRAFO_TRS', 'FAS_CON_S', 'TIP_CC', 'POSTO', 'CAP_CHA', 'FAS_CON_P', 'ARE_LOC', 'CONF'}


In [15]:
numerical = data.describe().columns

In [16]:
print(numerical)

Index(['TEN_LIN_SE', 'POT_NOM', 'PER_FER', 'PER_TOT', 'CAR_INST',
       'UCBT_ENE_MED', 'UCBT_ENE_STD', 'UCBT_ENE_MAX', 'NCON', 'NCON_URB',
       'CTMT_ENE_MED', 'CTMT_ENE_STD', 'CTMT_ENE_MAX', 'PERD_A3a', 'PERD_A4',
       'PERD_B', 'PERD_MED', 'PERD_A3a_B', 'PERD_A4_B', 'PERD_B_A3a',
       'PERD_B_A4', 'CTMT_PNTMT_MED', 'CTMT_PNTMT_STD', 'CTMT_PNTMT_MAX',
       'CTMT_PNTBT_MED', 'CTMT_PNTBT_STD', 'CTMT_PNTBT_MAX', 'POT_NOM_TRS',
       'POT_F01', 'POT_F02', 'PER_FER_TRS', 'PER_TOT_TRS', 'UNTRS_ENES_MED',
       'UNTRS_ENES_STD', 'UNTRS_ENES_MAX', 'UNTRS_ENET_MED', 'UNTRS_ENET_STD',
       'UNTRS_ENET_MAX', 'DEGREE', 'NEIG_DEGREE', 'BET_CEN', 'CLO_CEN',
       'PAGE_RANK', 'DISTANCE_SUB', 'FCONV_R', 'I_MAX_MED_R', 'I_MAX_MIN_R',
       'I_NOM_MED_R', 'I_NOM_MIN_R', 'REAC_SUB', 'RESI_SUB', 'Z_MOD_SUB',
       'Z_ANG_SUB'],
      dtype='object')


In [23]:
# Get Features from each origin
tabular_cols = set(tabular_ft.columns) & set(data.columns)
gmetrics_cols = set(gmetrics_ft.columns) & set(data.columns)
path_cols = set(path_ft.columns) & set(data.columns)

### Codificacion de variables categoricas

In [24]:
data_labeled = pd.get_dummies(data, columns=categorical, dummy_na= True, drop_first= True)

In [25]:
all_cols = set(data_labeled.columns)

In [26]:
tabular_lb_cols = [s for s in all_cols if any(xs in s for xs in tabular_cols & categorical)]

## Split Data sets

In [52]:
from sklearn.model_selection import train_test_split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(data_labeled, target, test_size=0.2, random_state=0)

## Generar Baselines con distintos tipo de grupos

In [54]:
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from xgboost.sklearn import XGBRegressor

from sklearn.model_selection import cross_val_score

In [98]:
models = []
models.append(("LinearRegression", Lasso()))
models.append(("KNN_Regression", KNeighborsRegressor()))
models.append(("XGBoost", XGBRegressor(n_estimators=300, n_jobs=-1)))

features = []
features.append(('Tabulares', tabular_lb_cols))
features.append(('Metricas Grafo', gmetrics_cols))
features.append(('Conexion Electrica', path_cols))
features.append(('Todas', all_cols))

In [107]:
experiment_id = mlflow.create_experiment("Baselines por conjunto de variables")
print(f"Experiment id: {experiment_id}")

Experiment id: 3


C:\Users\RamiroJC\AppData\Roaming\Python\Python37\site-packages\yaml\constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(key, collections.Hashable):


In [110]:
results_df = pd.DataFrame(columns= ['Regresion Lineal', 'Regresion KNN', 'Regresion XGBoost'])


for ft_group, ft_set in features:
    results_row = []

    for name, model in models:
        
        with mlflow.start_run(experiment_id = experiment_id):
            result = cross_val_score(model, X_train[ft_set], y_train, cv=3, scoring='neg_mean_absolute_error')
            results_row.append(abs(result.mean()))

            params = dict(model= name, 
                          feature_group= ft_group,
                          cv=3)

            mlflow.log_params(params)
            mlflow.log_metric('mean_absolute_error', abs(result.mean()))
            mlflow.sklearn.log_model(model, f'{name}_{ft_group}')

    results_df.loc[ft_type] = results_row

In [111]:
results_df

,Regresion Lineal,Regresion KNN,Regresion XGBoost
Todas,2.975346,2.433219,2.157823


## Seleccion de features

### Basada en modelado XGBoost

In [81]:
xb_selection = XGBRegressor(learning_rate=0.03,
                     n_estimators=1000,
                     max_depth=3,
                     subsample=0.8,
                     colsample_bytree=1,
                     gamma=1,
                     eval_metric='mae')

# Entreno sobre el logarimo de la variable de salidad
xb_selection.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='mae', gamma=1,
       importance_type='gain', learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

#### Seleccionar variables que afectan al error

In [82]:
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics

In [112]:
experiment_id = mlflow.create_experiment("Seleccion de variables por incremento de error")
print(f"Experiment id: {experiment_id}")

Experiment id: 4


In [119]:
# Aumento el requerimiento de importancia y calculo
# la cantidad de variables que permanecen y el error en un modelo de prueba

for thresh in [3e-3, 5e-3, 6e-3, 7e-3, 9e-3]:
    
    # select features using threshold
    selection = SelectFromModel(xb_selection, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    
    with mlflow.start_run(experiment_id = experiment_id):
        # Entrenar un modelo de prueba
        selection_model = XGBRegressor(learning_rate=0.1,
                         n_estimators= 300,
                         max_depth=3,
                         subsample=0.8,
                         colsample_bytree=1,
                         gamma=1,
                         eval_metric='mae')

        selection_model.fit(select_X_train, y_train)

        # Obtener metricas del modelo de prueba
        select_X_test = selection.transform(X_test)
        y_pred = selection_model.predict(select_X_test)

        error = metrics.mean_absolute_error(y_test, y_pred)
        
        mlflow.log_param('Umbral Importancia', thresh)
        mlflow.log_metric('mean_absolute_error', error)
        mlflow.log_metric('Cantidad de variables', select_X_train.shape[1])
        mlflow.sklearn.log_model(selection_model, f'xg_sel_tr{thresh:.3f}')

        print("Thresh=%.3f, n=%d, Error: %.2f" % (thresh, select_X_train.shape[1], error))

Thresh=0.003, n=70, Error: 2.03
Thresh=0.005, n=41, Error: 2.00
Thresh=0.006, n=32, Error: 1.96
Thresh=0.007, n=26, Error: 2.03
Thresh=0.009, n=18, Error: 2.08


In [86]:
indices = np.argsort(xb_selection.feature_importances_)[::-1]
top_features = X_train.columns[indices][:32]
top_features

Index(['Z_MOD_SUB', 'REAC_SUB', 'CTMT_PNTBT_MAX', 'CTMT_PNTMT_STD',
       'ARE_LOC_UB', 'PERD_A4_B', 'RESI_SUB', 'I_MAX_MED_R', 'CTMT_ENE_STD',
       'CTMT_PNTBT_MED', 'NCON_URB', 'CTMT_PNTMT_MAX', 'UNTRS_ENES_MED',
       'FAS_CON_S_BN', 'PER_TOT', 'FCONV_R', 'CLO_CEN', 'CTMT_ENE_MED',
       'PERD_A4', 'PERD_B', 'CTMT_PNTMT_MED', 'TIP_TRAFO_T', 'UCBT_ENE_MED',
       'DISTANCE_SUB', 'BET_CEN', 'CAP_ELO_2H', 'POT_NOM_TRS', 'GRU_TAR_B1',
       'TIP_CC_RES-Tipo10', 'FAS_CON_T_NULL', 'CTMT_ENE_MAX', 'Z_ANG_SUB'],
      dtype='object')

#### Chequear baseline de modelo con seleccion de variables

In [120]:
mlflow.set_experiment("Baselines por conjunto de variables")

In [121]:
results_row = []

for name, model in models:
    with mlflow.start_run():
        result = cross_val_score(model, X_train[top_features], y_train, cv=3, scoring='neg_mean_absolute_error')
        results_row.append(abs(result.mean()))
        
        params = dict(model= name, 
                      feature_group= 'Seleccion XGB',
                      cv=3)

        mlflow.log_params(params)
        mlflow.log_metric('mean_absolute_error', abs(result.mean()))
        mlflow.sklearn.log_model(model, f'{name}_{ft_group}')
    
results_df.loc['Seleccion Variables'] = results_row

In [102]:
results_df

,Regresion Lineal,Regresion KNN,Regresion XGBoost
Tabulares,4.880642,4.012538,3.734812
Metricas Grafo,5.751176,4.192741,3.379558
Conexion Electrica,3.436514,2.825810,2.500436
Todas,2.975346,2.433219,2.157823
Seleccion Variables,3.074119,2.447425,2.127191


## Exportar Exit Point

In [103]:
filename = './data/3_FtSet_XGImportance.pkl'
outfile = open(filename, 'wb')
pickle.dump(data_labeled[top_features], outfile)
outfile.close()

In [105]:
filename = './data/3_Target.pkl'
outfile = open(filename, 'wb')
pickle.dump(target, outfile)
outfile.close()